In [1]:
from sqlalchemy import create_engine, inspect, ForeignKey, Column, Integer, Float, String, and_, null

from sqlalchemy.orm import relationship, remote, backref
from sqlalchemy.orm.collections import attribute_mapped_collection

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
class NetworkModel(Base):
    __tablename__ = 'models'

    id          = Column(Integer, primary_key = True, autoincrement=True)
    name        = Column(String, nullable=False)
    description = Column(String)
    
    wbs        = relationship('WBSRecord', backref='model', cascade = 'all, delete-orphan')
    wbs_root   = relationship('WBSRecord', viewonly=True,
                              primaryjoin='and_(WBSRecord.model_id == NetworkModel.id, WBSRecord.parent_id == null())')
    
    actitivies = relationship('Activity' , backref='model', cascade = 'all, delete-orphan')
    links      = relationship('Link'     , backref='model', cascade = 'all, delete-orphan')
    events     = relationship('Event'    , backref='model', cascade = 'all, delete-orphan')
    
    #----------------------------------------------------------------------------------------------
    def __init__(self, session, name, description=''):
        self.name        = name
        self.description = description

        session.add(self)
        session.flush() #We've got an id now

    #----------------------------------------------------------------------------------------------
    def describe(self):
        print(self.id, ':', self.name, ':', self.description)
    
    #----------------------------------------------------------------------------------------------
    def copy(self, name=None):
        if not name:
            cp = NetworkModel(inspect(self).session, 
                              description=self.description, 
                              name='Copy of: ' + self.name)
        else:
            cp = NetworkModel(inspect(self).session,
                              description=self.description, 
                              name=name)
        for w in self.wbs_root:
            w.copy(model=cp)
        
        cp.links  = [l.copy(model=cp) for l in self.links]
        cp.events = [e.copy(model=cp) for e in self.events]

        inspect(self).session.flush()
        return cp
    
    #----------------------------------------------------------------------------------------------
    @property
    def _iwbs(self):
        return dict([(w.path, i) for i,w in enumerate(self.wbs)])
    
    def wbs_record(self, path):
        return self.wbs[self._iwbs[path]]
    
    #----------------------------------------------------------------------------------------------
    def delete_wbs_record(self, path):
        w = self.wbs.pop(self._iwbs[path])
        del w
        inspect(self).session.flush()

###################################################################################################
class WBSRecord(Base):
    __tablename__ = 'wbs'

    id        = Column(Integer, primary_key=True, autoincrement=True)
    model_id  = Column(Integer, ForeignKey('models.id'))
    parent_id = Column(Integer, ForeignKey('wbs.id'))
    
    path      = Column(String, nullable=False, index=True)
    name      = Column(String, nullable=False)
    
    children = relationship("WBSRecord",
                            remote_side=parent_id,
                            back_populates='parent',
                            cascade='all'
                           )
    
    parent  = relationship("WBSRecord",
                            remote_side=id,
                            back_populates='children',
                           )
    
    activity = relationship('Activity', 
                            backref='wbs', 
                            cascade = 'all, delete-orphan', 
                            uselist=False)
    
    #----------------------------------------------------------------------------------------------
    def __init__(self, name, parent=None, model=None):
        if parent:
            assert isinstance(parent, WBSRecord)
            model = parent.model

        assert isinstance(model, NetworkModel)

        self.name   = name
        self.parent = parent
        self.model  = model
        self.path   = ''

        #That's shitty approach but I don't know how's better...
        inspect(self).session.flush()

        if parent:
            self.path += parent.path + '.'
        self.path += str(self.id)

    #----------------------------------------------------------------------------------------------
    def _propagate_path(self, new_path_base):
        if self.parent:
            self.path = self.parent.path + '.' + str(self.id)
        else:
            self.path = str(self.id)

        for c in self.children:
            c._propagate_path(self.path)

    #----------------------------------------------------------------------------------------------
    def move_to(self, new_path_base):
        if new_path_base != '':
            if new_path_base.startswith(self.path):
                raise ValueError('WTF R U doing motherfucker???')
            self.parent = self.model.comment(new_path_base)
        else:
            self.parent = None

        self._propagate_path(new_path_base)
        inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    def copy(self, parent=None, model=None):
        cp = WBSRecord(self.name, parent=parent, model=model)

        if isinstance(self.activity, Activity):
            cp.activity = self.activity.copy(cp)

        cp.children = [c.copy(parent=cp) for c in self.children]

        return cp
    
    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'WBSRecord(model_id=%r id=%r, path=%r, name=%r)' % (
            self.model_id,
            self.id,
            self.path,
            self.name
        )

    #----------------------------------------------------------------------------------------------
    def dump(self, _level=0):
        return (
                '   ' * _level
                + repr(self)
                + "\n"
                + "".join([c.dump(_level + 1) for c in self.children])
        )

###################################################################################################
class Activity(Base):
    __tablename__ = 'activities'

    
    model_id = Column(Integer, ForeignKey('models.id'), primary_key=True)
    id       = Column(Integer,                          primary_key=True)
    
    wbs_id   = Column(Integer, ForeignKey('wbs.id'))
        
    in_links  = relationship('Link', 
                             primaryjoin='and_(Activity.model_id == Link.model_id, Activity.id == Link.src_id)',
                             backref='src', cascade = 'all, delete-orphan')
    out_links = relationship('Link', 
                             primaryjoin='and_(Activity.model_id == Link.model_id, Activity.id == Link.dst_id)', 
                             backref='dst', cascade = 'all, delete-orphan')

    #Работы являются связями для событий
    src_id    = Column(Integer, ForeignKey('events.id'))
    dst_id    = Column(Integer, ForeignKey('events.id'))
    
    #CPM Data
    duration    = Column(Float, default=0.0) # Длительность работы
    early_start = Column(Float, default=0.0) # Ранний старт
    late_start  = Column(Float, default=0.0) # Поздний старт
    early_end   = Column(Float, default=0.0) # Ранний финиш
    late_end    = Column(Float, default=0.0) # Поздний финиш
    reserve     = Column(Float, default=0.0) # Резерв времени
    
    def __init__(self, wbs, id=None, src=None, dst=None, src_id=None, dst_id=None,):
        assert isinstance(wbs, WBSRecord)

        if src:
            assert isinstance(src, Event)
            src_id = src.id

        if dst:
            assert isinstance(dst, Event)
            dst_id = dst.id

        self.model  = wbs.model
        #Обеспечили уникальность при генерации и возможность копирования
        self.id     = id if id else wbs.id
        self.wbs    = wbs
        self.src_id = src_id
        self.dst_id = dst_id

        inspect(self).session.flush()

    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Activity(model_id=%r id=%r, path=%r, name=%r, src_id=%r, dst_id=%r)' % (
            self.model_id,
            self.id,
            self.wbs.path,
            self.wbs.name,
            self.src_id,
            self.dst_id
        )
    
    #----------------------------------------------------------------------------------------------
    def copy(self, wbs):
        return Activity(wbs, src=self.src, dst=self.dst, id=self.id)

###################################################################################################
class Link(Base):
    __tablename__ = 'links'

    model_id  = Column(Integer, ForeignKey('models.id')    ,  primary_key=True)
    src_id    = Column(Integer, ForeignKey('activities.id'),  primary_key=True)
    dst_id    = Column(Integer, ForeignKey('activities.id'),  primary_key=True)
    
    def __init__(self, model=None, src=None, dst=None, src_id=None, dst_id=None):

        if model:
            assert isinstance(model, NetworkModel)
        
        if src:
            assert isinstance(src, Activity)
            src_id = src.id
            
        if dst:
            assert isinstance(dst, Activity)
            dst_id = dst.id
            
        if src and dst:
            assert src.model_id == dst.model_id
            if model:
                assert src.model_id == model.id
            else:
                model = src.model
                
        self.model_id = model.id
        self.src_id   = src_id
        self.dst_id   = dst_id
            
    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Link(model_id=%r src_id=%r, dst_id=%r)' % (
            self.model_id,
            self.src_id,
            self.dst_id
        )
    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Link(src=self.src, dst=self.dst, model=model)

###################################################################################################
#TODO: 4. Добавить узлы и ребра для визуализации сетевой модели.
class Event(Base):
    __tablename__ = 'events'

    id       = Column(Integer,                          primary_key=True)
    model_id = Column(Integer, ForeignKey('models.id'), primary_key=True)

    in_activities  = relationship('Activity', 
                             primaryjoin='and_(Event.model_id == Activity.model_id, Event.id == Activity.src_id)',
                             backref='src')
    out_activities = relationship('Activity', 
                             primaryjoin='and_(Event.model_id == Activity.model_id, Event.id == Activity.dst_id)', 
                             backref='dst')
    
    early   = Column(Float, default=0.0)
    late    = Column(Float, default=0.0)
    reserve = Column(Float, default=0.0)

    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Event(model_id=%r id=%r)' % (
            self.model_id,
            self.id
        )

    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Event(model=model,id=self.id)

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo = True)

In [4]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
Base.metadata.create_all(engine)

2021-08-30 22:05:56,129 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-30 22:05:56,130 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("models")
2021-08-30 22:05:56,130 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-30 22:05:56,131 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("models")
2021-08-30 22:05:56,131 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-30 22:05:56,132 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("wbs")
2021-08-30 22:05:56,132 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-30 22:05:56,133 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("wbs")
2021-08-30 22:05:56,133 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-30 22:05:56,134 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("activities")
2021-08-30 22:05:56,134 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-30 22:05:56,134 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("activities")
2021-08-30 22:05:56,135 INFO sqlalchemy.engine.Engine [raw sql] ()
202

In [6]:
nm = NetworkModel(session, name='First model!')

2021-08-30 22:05:56,267 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-30 22:05:56,269 INFO sqlalchemy.engine.Engine INSERT INTO models (name, description) VALUES (?, ?)
2021-08-30 22:05:56,269 INFO sqlalchemy.engine.Engine [generated in 0.00052s] ('First model!', '')


In [7]:
nm.describe()

1 : First model! : 


In [8]:
w1 = WBSRecord('Test 1', model=nm)
a1 = Activity(w1)

2021-08-30 22:05:56,665 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-08-30 22:05:56,665 INFO sqlalchemy.engine.Engine [generated in 0.00076s] (1, None, '', 'Test 1')
2021-08-30 22:05:56,672 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities 
WHERE ? = activities.wbs_id
2021-08-30 22:05:56,673 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (1,)
2021-08-30 22:05:56,676 INFO sqlalchemy.engine.Engine UPDATE wbs SET path=? WHERE wbs.id = ?
2021-08-30 22:05:56,67

In [9]:
w2 = WBSRecord('Test 2', model=nm)
w3 = WBSRecord('Test 3', parent=w1)
a3 = Activity(w3)

l1 = Link(src=a1, dst=a3)

ev1 = Event(model=nm, id=1)
ev2 = Event(model=nm, id=2)
ev3 = Event(model=nm, id=3)
print(session.query(Activity).all())

2021-08-30 22:05:56,760 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-08-30 22:05:56,760 INFO sqlalchemy.engine.Engine [cached since 0.09546s ago] (1, None, '', 'Test 2')
2021-08-30 22:05:56,762 INFO sqlalchemy.engine.Engine UPDATE wbs SET path=? WHERE wbs.id = ?
2021-08-30 22:05:56,762 INFO sqlalchemy.engine.Engine [cached since 0.08684s ago] ('2', 2)
2021-08-30 22:05:56,764 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-08-30 22:05:56,764 INFO sqlalchemy.engine.Engine [cached since 0.09959s ago] (1, 1, '', 'Test 3')
2021-08-30 22:05:56,766 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activiti

In [10]:
a1.src = ev1
a1.dst = ev2

a3.src = ev2
a3.dst = ev3
print(session.query(Activity).all())

2021-08-30 22:05:56,924 INFO sqlalchemy.engine.Engine UPDATE activities SET src_id=?, dst_id=? WHERE activities.model_id = ? AND activities.id = ?
2021-08-30 22:05:56,924 INFO sqlalchemy.engine.Engine [generated in 0.00068s] ((1, 2, 1, 1), (2, 3, 1, 3))
2021-08-30 22:05:56,926 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities
2021-08-30 22:05:56,926 INFO sqlalchemy.engine.Engine [cached since 0.1468s ago] ()
[Activity(model_id=1 id=1, path='1', name='Test 1', src_id=1, dst_id=2), Activity(model_id=1 id=3, path='1.3', name

In [11]:
print(nm.wbs_root)

2021-08-30 22:05:57,025 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs 
WHERE wbs.model_id = ? AND wbs.parent_id IS NULL
2021-08-30 22:05:57,026 INFO sqlalchemy.engine.Engine [generated in 0.00060s] (1,)
[WBSRecord(model_id=1 id=1, path='1', name='Test 1'), WBSRecord(model_id=1 id=2, path='2', name='Test 2')]


In [12]:
print(nm.wbs)

2021-08-30 22:05:57,169 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs 
WHERE ? = wbs.model_id
2021-08-30 22:05:57,170 INFO sqlalchemy.engine.Engine [generated in 0.00079s] (1,)
[WBSRecord(model_id=1 id=1, path='1', name='Test 1'), WBSRecord(model_id=1 id=2, path='2', name='Test 2'), WBSRecord(model_id=1 id=3, path='1.3', name='Test 3')]


In [13]:
nm2 = nm.copy()

2021-08-30 22:05:57,267 INFO sqlalchemy.engine.Engine INSERT INTO models (name, description) VALUES (?, ?)
2021-08-30 22:05:57,268 INFO sqlalchemy.engine.Engine [cached since 0.9992s ago] ('Copy of: First model!', '')
2021-08-30 22:05:57,269 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-08-30 22:05:57,270 INFO sqlalchemy.engine.Engine [cached since 0.6054s ago] (2, None, '', 'Test 1')
2021-08-30 22:05:57,272 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities 
WHERE

In [14]:
print(nm2.wbs)

2021-08-30 22:05:57,366 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs 
WHERE ? = wbs.model_id
2021-08-30 22:05:57,367 INFO sqlalchemy.engine.Engine [cached since 0.1986s ago] (2,)
[WBSRecord(model_id=2 id=4, path='4', name='Test 1'), WBSRecord(model_id=2 id=5, path='4.5', name='Test 3'), WBSRecord(model_id=2 id=6, path='6', name='Test 2')]


In [15]:
from sqlalchemy import select, text
print(session.query(WBSRecord).all())

2021-08-30 22:05:57,499 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs
2021-08-30 22:05:57,500 INFO sqlalchemy.engine.Engine [generated in 0.00058s] ()
[WBSRecord(model_id=1 id=1, path='1', name='Test 1'), WBSRecord(model_id=1 id=2, path='2', name='Test 2'), WBSRecord(model_id=1 id=3, path='1.3', name='Test 3'), WBSRecord(model_id=2 id=4, path='4', name='Test 1'), WBSRecord(model_id=2 id=5, path='4.5', name='Test 3'), WBSRecord(model_id=2 id=6, path='6', name='Test 2')]


In [16]:
print(session.query(Activity).all())

2021-08-30 22:05:57,632 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities
2021-08-30 22:05:57,632 INFO sqlalchemy.engine.Engine [cached since 0.8529s ago] ()
[Activity(model_id=1 id=1, path='1', name='Test 1', src_id=1, dst_id=2), Activity(model_id=1 id=3, path='1.3', name='Test 3', src_id=2, dst_id=3), Activity(model_id=2 id=1, path='4', name='Test 1', src_id=1, dst_id=2), Activity(model_id=2 id=3, path='4.5', name='Test 3', src_id=2, dst_id=3)]


In [17]:
print(session.query(Link).all())

2021-08-30 22:05:57,710 INFO sqlalchemy.engine.Engine SELECT links.model_id AS links_model_id, links.src_id AS links_src_id, links.dst_id AS links_dst_id 
FROM links
2021-08-30 22:05:57,710 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ()
[]


In [18]:
print(session.query(Event).all())

2021-08-30 22:05:57,843 INFO sqlalchemy.engine.Engine SELECT events.id AS events_id, events.model_id AS events_model_id, events.early AS events_early, events.late AS events_late, events.reserve AS events_reserve 
FROM events
2021-08-30 22:05:57,843 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ()
[Event(model_id=1 id=1), Event(model_id=1 id=2), Event(model_id=1 id=3), Event(model_id=2 id=1), Event(model_id=2 id=2), Event(model_id=2 id=3)]
